In [1]:
!pip install -U transformers torch bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Load model and tokenizer
model_name = "microsoft/Phi-3-mini-4k-instruct"

# 4-bit Quantization Configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # Efficient dtype
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [36]:
def generate_few_shot_prompt(english_sentences, indonesian_targets, new_sentence):
    """
    Generate a few-shot prompt for English to Indonesian translation.

    Parameters:
        english_sentences (list): List of example English sentences.
        indonesian_targets (list): List of corresponding Indonesian translations.
        new_sentence (str): The new English sentence to translate.

    Returns:
        str: Formatted few-shot prompt.
    """
    # Validate inputs
    if len(english_sentences) != len(indonesian_targets):
        raise ValueError("The number of English sentences and Indonesian targets must match.")

    # Start with "Examples"
    examples = "Example:\n"
    for eng, indo in zip(english_sentences, indonesian_targets):
        examples += f"English: {eng}\nIndonesian: {indo}\n\n"

    # Add the new sentence for translation
    prompt = (
        # "Translate the following English sentences to Indonesian:\n\n" +
        examples
        + f"Translate the following English sentence to Indonesian.\nEnglish: {new_sentence}\nIndonesian:"
    )

    return prompt


In [182]:
# Test Data Examples
# english_sentences =
#     "Can I help you?"
#     "As you can see, I'm jet-lagged and trying to adjust to the time-zone change."
#     "That's what the policeman just said."
#     "Yeah, we've been teaching him, but really, he's been teaching us."
#     "Someone wants to kill him."
#     "Hey Four, that's pretty good right?"
#     "You should've told me, boy."
#     "Your bed is in the corner, over there."
#     "Hey, kid, you've done really good so far, so what we're gonna do right now is we're gonna call your daddy."

# indonesian_targets =
#     "ada yang bisa kubantu?"
#     "Seperti yang kamu lihat , aku alami jet - lag dan mencoba untuk menyesuaikan diri dengan perubahan zona waktu ."
#     "Begitulah kata polisinya tadi."
#     "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
#     "Seseorang ingin membunuhnya."
#     "Four, Tak begitu buruk, kan?"
#     "Kau harusnya katakan padaku, Nak."
#     "Ranjangmu disudut itu."
#     "Hei, nak, sejauh ini kau telah melakukannya dengan baik sekali, jadi apa yang akan kita lakukan sekarang adalah kita akan menelpon ayahmu."



# Training Data Examples
english_sentences = [
    "I feel sorry for her.",
    # "I am always with you.",
    # "We didn't understand much of what he said, only that he was suffering.",
    # "Hey, good morning!",
    # "That's never happened.",
    # "This capsule hits the water and dissolves, the sodium'll give off hydrogen, and there'll enough of a reaction to ignite it.",
    # "It's just, the way I asked you, I kind of left out the most important part."
]

indonesian_targets = [
    "Aku merasa kasihan padanya.",
    # "Saya akan selalu bersamamu.",
    # "Kami tidak mengerti apa yang ia katakan, selain bahwa dia menderita.",
    # "Hei, selamat pagi!",
    # "Itu tidak pernah terjadi.",
    # "kapsul ini akan menyentuh air dan larut, sodium akan membentuk hidrogen, dan reaksi yang cukup akan menyalakannya.",
    # "Hanya saja, caraku menanyakannya padamu, aku agak merasa kelewatan bagian yang paling pentingnya."
]

# The new sentence to translate
new_sentence = "Can I help you?"

# Generate the prompt
prompt = generate_few_shot_prompt(english_sentences, indonesian_targets, new_sentence)
print(prompt)

Example:
English: I feel sorry for her.
Indonesian: Aku merasa kasihan padanya.

Translate the following English sentence to Indonesian.
English: Can I help you?
Indonesian:


In [184]:
# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)

# Generate the response
output = model.generate(
      input_ids=inputs,
      max_new_tokens=60,
      do_sample=True,
      top_k=10,
      top_p=0.9,
      temperature=0.3,
      repetition_penalty=1.5,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )
response = tokenizer.decode(output[0], skip_special_tokens=True)

print(response)

Example:
English: I feel sorry for her.
Indonesian: Aku merasa kasihan padanya.

Translate the following English sentence to Indonesian.
English: Can I help you?
Indonesian: Bisakah saya membantu andai?. 


In [125]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure the nltk 'punkt' tokenizer is downloaded
nltk.download('punkt_tab')

def calculate_bleu_single_reference(reference_sentence, candidate_sentence):
    """
    Calculate the BLEU score for a single predicted translation against one reference.

    Parameters:
        reference_sentence (str): The correct reference translation.
        candidate_sentence (str): The predicted translation.

    Returns:
        float: The BLEU score.
    """
    # Tokenize the reference sentence and candidate sentence
    tokenized_reference = [nltk.word_tokenize(reference_sentence)]
    tokenized_candidate = nltk.word_tokenize(candidate_sentence)

    # Use smoothing to handle short sentences
    smoothing_function = SmoothingFunction().method1

    # Calculate BLEU score
    bleu_score = sentence_bleu(tokenized_reference, tokenized_candidate, smoothing_function=smoothing_function)
    return bleu_score

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [185]:
# Test Data Examples
# english_sentences =
#     "Can I help you?"
#     "As you can see, I'm jet-lagged and trying to adjust to the time-zone change."
#     "That's what the policeman just said."
#     "Yeah, we've been teaching him, but really, he's been teaching us."
#     "Someone wants to kill him."
#     "Hey Four, that's pretty good right?"
#     "You should've told me, boy."
#     "Your bed is in the corner, over there."
#     "Hey, kid, you've done really good so far, so what we're gonna do right now is we're gonna call your daddy."

# indonesian_targets =
#     "ada yang bisa kubantu?"
#     "Seperti yang kamu lihat , aku alami jet - lag dan mencoba untuk menyesuaikan diri dengan perubahan zona waktu ."
#     "Begitulah kata polisinya tadi."
#     "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
#     "Seseorang ingin membunuhnya."
#     "Four, Tak begitu buruk, kan?"
#     "Kau harusnya katakan padaku, Nak."
#     "Ranjangmu di sudut itu."
#     "Hei, nak, sejauh ini kau telah melakukannya dengan baik sekali, jadi apa yang akan kita lakukan sekarang adalah kita akan menelpon ayahmu."

# Single reference translation
reference_sentence = "ada yang bisa kubantu?"
candidate_sentence = "Bisakah saya membantu andai?."

# Calculate BLEU score
bleu_score = calculate_bleu_single_reference(reference_sentence, candidate_sentence)
print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.0408
